In [1]:
import torch.distributions.distribution as D
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import data.read_samples as rs
import torch.optim as optim
import torch.utils.data
import torch.nn as nn
import numpy as np
import datetime
import torch
import time
import os

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 50
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

[INFO] Pre-processing for make clean
2022-05-14 14:45:39.244250 model.py code start


In [2]:
device = torch.device('cuda')
print(torch.cuda.get_device_name(device))
os.environ['KMP_DUPLICATE_LIB_OK']='True'

NVIDIA GeForce RTX 3080 Laptop GPU


In [3]:
class RBM(nn.Module): 
    with torch.cuda.device(0):
        def __init__(self, n_vis, n_hid, k, batch):
            super(RBM, self).__init__()
            self.W      = nn.Parameter(torch.randn(1, batch, device=device) * 1e-2)
            self.n_vis  = n_vis
            self.n_hid  = n_hid
            self.k      = k
            self.batch  = batch
            self.v_bias = nn.Parameter(torch.zeros(batch, device=device))
            self.h_bias = nn.Parameter(torch.zeros(batch, device=device))
        
        def sample_from_p(self, p):
            return F.relu(
                torch.sign(
                    p - Variable(torch.randn(p.size(), device=device))
                )
            ).to(device=device)

        ''' ISSUE PART '''
        def v_to_h(self, v):
            w = (self.W.clone())

            p_h = F.sigmoid(
                F.linear(v, w)
            ).to(device=device)

            sample_h = self.sample_from_p(p_h)
            return p_h, sample_h

        def h_to_v(self, h):
            w = self.W.t().clone()

            p_v = F.sigmoid(
                F.linear(h, w).to(device=device)
            ).to(device=device)

            sample_v = self.sample_from_p(p_v)
            return p_v, sample_v
        
        def forward(self, v):
            pre_h1, h1 = self.v_to_h(v)
            h_ = h1

            for _ in range(self.k):
                pre_v_, v_ = self.h_to_v(h_)
                pre_h_, h_ = self.v_to_h(v_)
            return v, v_
        
        def get_weight(self):
            return self.W

In [ ]:
class SVM(nn.Module):
    def __init__ (self, epoch, n_feat, n_out, batch=10, lr=0.999, c=0.01):
        super(SVM, self).__init__()
        self.epoch = epoch
        self.n_feat = n_feat
        self.n_out = n_out
        self.batch = batch
        self.lr = lr
        self.c = c

    def get_accuracy(self, model, data):
            loader = torch.utils.data.DataLoader(data, batch_size=self.batch)
            correct, total = 0, 0

            for xs, ts in loader:
                zs = model(xs)
                pred = zs.max(1, keepdim=True)[1]
                correct += pred.eq(ts.view_as(pred)).sum().item()
                total += int(ts.shape[0])
                return correct / total        
    
    def plot(self, xl, yl, xls, yls, label, title="Linear SVM Model Result"):
        plt.title(title)
        plt.plot(xl, yl, label)
        plt.xlabel("Iterations")
        plt.ylabel("Loss")
        plt.show()
        
        plt.title("Training Curve (batch_size={}, lr={})".format(self.batch, self.lr))
        plt.plot(xls, yls, label="Train")
        plt.xlabel("Iterations")
        plt.ylabel("Accuracy")
        plt.legend(loc='best')
        plt.show()

    def train(self, x):
        iters, loss_ = [], []
        iters_sub, train_acc = [], []
        
        model = nn.Linear(self.n_feat, self.n_out)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adagrad(model.parameters(), lr=self.lr, weight_decay=self.c)
        
        weight, bias = list(model.parameters())
        y = torch.sigmoid(model(x))
        print("weight shape: {}\tbias shape: {}".format(weight.shape, bias.shape))

        svm_train_dataloader = DataLoader(x,
                                          batch_size=self.batch,
                                          shuffle=True)
        
        n = 0
        for epoch in range(self.epoch):
            for xs, ts in iter(svm_train_dataloader):
                if len(ts) != self.batch:
                    continue
                zs = model(xs)

                loss = criterion(zs, ts)
                loss.backward()
                
                optimizer.step()
                optimizer.zero_grad()

                iters.append(n)
                loss_.append(float(loss) / self.batch)
                train_acc.append(self.get_accuracy(model, x))
                n += 1
        
        self.plot(iters, loss_, iters_sub, train_acc, label="Train")
        return model

In [6]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
db1_sig, db1_label, db2_sig, db2_label, db3_sig, db3_label = rs.return_list()

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read records file from  ./data/db1/
[RSLT]			 Export records ...
		 ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124']
[INFO]./rdsamp commending start
[IWIP]		rdsamp Converting 100
[IWIP]		rdsamp Converting 101
[IWIP]		rdsamp Converting 102
[IWIP]		rdsamp Converting 103
[IWIP]		rdsamp Converting 104
[IWIP]		rdsamp Converting 105
[IWIP]		rdsamp Converting 106
[IWIP]		rdsamp Converting 107
[IWIP]		rdsamp Converting 108
[IWIP]		rdsamp Converting 109
[IWIP]		rdsamp Converting 111
[IWIP]		rdsamp Converting 112
[IWIP]		rdsamp Converting 113
[IWIP]		rdsamp Converting 114
[IWIP]		rdsamp Converting 115
[IWIP]		rdsamp Converting 116
[IWIP]		rdsamp Converting 117
[IWIP]		rdsamp Converting 118
[IWIP]		rdsamp Converting 119
[IWIP]		rdsamp Converting 121
[IWIP

In [7]:
train_dataloader = DataLoader(db1_sig + db2_sig,
                              batch_size=BATCH_SIZE,
                              num_workers=0, 
                              collate_fn=lambda x: x,
                              shuffle=True)
                              
test_dataloader = DataLoader(db2_sig + db3_sig,
                             batch_size=BATCH_SIZE,
                             num_workers=0, 
                             collate_fn=lambda x: x,
                             shuffle=True)

In [8]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

omse_loss = list()
mse_loss = nn.MSELoss()
best_acc = float()

In [9]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    run_acc = float()
    start = time.time()
    '''First bbrbm'''
    for i, (data) in enumerate(train_dataloader):
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)
        
        sample_data = torch.bernoulli(data).view(-1, 10).to(device=device)

        # tensor binary
        vog_first, v1 = rbm_first(sample_data)
        omse_loss = mse_loss(vog_first, v1)

        first_train_op.zero_grad()
        first_train_op.step()
        omse_loss.backward()
        run_acc += (torch.bernoulli(data).view(-1, 10).to(device=device) == v1).sum().item()
    
    acc = (run_acc) / v1.size()[0]
    if acc > best_acc:
        best_acc = acc

    path = "./saveMode_BBRBM1.pth"
    torch.save(rbm_first.state_dict(), path)

    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}\tAcc : {3}\tBest Acc : {4}" \
        .format(epoch + 1, 
                omse_loss, 
                time.time() - start,
                acc, 
                best_acc))

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2432/2333133579.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  torch.tensor(data, dtype=torch.float32)
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1ST BBrbm_first Training loss for 1 epoch 0.4999277889728546	Estimate time : 4.982951402664185	Acc : 29.99914903846154	Best Acc : 29.99914903846154
1ST BBrbm_first Training loss for 2 epoch 0.4996946156024933	Estimate time : 4.461601257324219	Acc : 30.0045625	Best Acc : 30.0045625
1ST BBrbm_first Training loss for 3 epoch 0.5002858638763428	Estimate time : 4.598857164382935	Acc : 29.99624903846154	Best Acc : 30.0045625
1ST BBrbm_first Training loss for 4 epoch 0.49988511204719543	Estimate time : 4.623755931854248	Acc : 30.000025	Best Acc : 30.0045625
1ST BBrbm_first Training loss for 5 epoch 0.49983394145965576	Estimate time : 7.552161693572998	Acc : 29.998073076923077	Best Acc : 30.0045625
1ST BBrbm_first Training loss for 6 epoch 0.5001282691955566	Estimate time : 4.670748949050903	Acc : 30.00035673076923	Best Acc : 30.0045625
1ST BBrbm_first Training loss for 7 epoch 0.500183641910553	Estimate time : 4.572844743728638	Acc : 29.99824423076923	Best Acc : 30.0045625
1ST BBrbm_first Tra

In [10]:
output_from_first = torch.tensor(output_from_first)
print(output_from_first.size())
run_acc = float()

for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    start = time.time()
    for _, (data) in enumerate(output_from_first): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, 10).to(device=device)

        # tensor binary
        vog_second, v2 = rbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        second_train_op.zero_grad()
        omse_loss.backward()
        second_train_op.step()
        run_acc += (torch.bernoulli(data).view(-1, 10).to(device=device) == v2).sum().item()

    acc = run_acc / v2.size()[0]
    if acc > best_acc:
        best_acc = acc

    path = "./saveMode_BBRBM_2.pth"
    torch.save(rbm_second.state_dict(), path)

    output_from_second.append(v2.tolist())
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}\tAcc : {3}\tBest Acc : {4}" \
        .format(epoch + 1, 
                omse_loss, 
                time.time() - start,
                acc, 
                best_acc))

torch.Size([50, 1040000, 10])


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2432/2345479556.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)


2ST BBrbm_first Training loss for 1 epoch 0.5000119209289551	Estimate time : 11.987833738327026	Acc : 280.0000086538462	Best Acc : 280.0000086538462
2ST BBrbm_first Training loss for 2 epoch 0.4999728798866272	Estimate time : 12.480892419815063	Acc : 530.0047605769231	Best Acc : 530.0047605769231
2ST BBrbm_first Training loss for 3 epoch 0.4998241364955902	Estimate time : 12.44279956817627	Acc : 780.0191509615385	Best Acc : 780.0191509615385
2ST BBrbm_first Training loss for 4 epoch 0.4999432861804962	Estimate time : 22.908244132995605	Acc : 1030.0136759615384	Best Acc : 1030.0136759615384
2ST BBrbm_first Training loss for 5 epoch 0.4999239444732666	Estimate time : 13.259594917297363	Acc : 1280.0059259615384	Best Acc : 1280.0059259615384
2ST BBrbm_first Training loss for 6 epoch 0.49956172704696655	Estimate time : 12.682008028030396	Acc : 1529.998714423077	Best Acc : 1529.998714423077
2ST BBrbm_first Training loss for 7 epoch 0.5001161694526672	Estimate time : 12.321842193603516	Acc : 

In [11]:
output_from_second = torch.tensor(output_from_second)
run_acc = float()

for epoch in range(EPOCH):
    '''Third bbrbm'''
    for _, (data) in enumerate(output_from_second):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, 10).to(device=device)

        vog_third, v3 = rbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3)
        
        third_train_op.zero_grad()
        omse_loss.backward()
        third_train_op.step()
        run_acc += (torch.bernoulli(data).view(-1, 10).to(device=device) == v3).sum().item()

    acc = run_acc / v3.size()[0]
    if acc > best_acc:
        best_acc = acc

    path = "./saveMode_BBRBM_3.pth"
    torch.save(rbm_third.state_dict(), path)

    output_from_third.append(v3.tolist())
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}\tAcc : {3}\tBest Acc : {4}" \
        .format(epoch + 1, 
                omse_loss, 
                time.time() - start,
                acc, 
                best_acc))


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2432/3636279507.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


3ST BBrbm_first Training loss for 1 epoch 0.5000220537185669	Estimate time : 0.5958292484283447	Acc : 12779.884936538461	Best Acc : 12779.884936538461
3ST BBrbm_first Training loss for 2 epoch 0.4999166429042816	Estimate time : 0.5550384521484375	Acc : 13029.91390096154	Best Acc : 13029.91390096154
3ST BBrbm_first Training loss for 3 epoch 0.4996604025363922	Estimate time : 9.588981866836548	Acc : 13279.928365384616	Best Acc : 13279.928365384616
3ST BBrbm_first Training loss for 4 epoch 0.49996432662010193	Estimate time : 0.5449974536895752	Acc : 13529.926442307693	Best Acc : 13529.926442307693
3ST BBrbm_first Training loss for 5 epoch 0.5000404715538025	Estimate time : 0.534782886505127	Acc : 13779.937292307692	Best Acc : 13779.937292307692
3ST BBrbm_first Training loss for 6 epoch 0.49995192885398865	Estimate time : 0.5790023803710938	Acc : 14029.945975	Best Acc : 14029.945975
3ST BBrbm_first Training loss for 7 epoch 0.4999968409538269	Estimate time : 0.5172364711761475	Acc : 14279.

In [12]:
# rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
# rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
# rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

output_from_first = list()
output_from_second = list()

omse_loss = list()
mse_loss = nn.MSELoss()
gaussian_std = torch.arange(1, 0, -0.1)

In [14]:
''' ** ISSUE PART ** '''

loss_ = []
run_acc = float()

for epoch in range(EPOCH):
    '''First gbrbm'''
    for i, (data) in enumerate(output_from_third):
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)
        
        sample_data = torch.normal(mean=data, std=gaussian_std).view(-1, 10).to(device=device)

        # tensor binary
        vog_first, v1 = rbm_first(sample_data)
        omse_loss = mse_loss(vog_first, v1)

        first_train_op.zero_grad()
        first_train_op.step()
        omse_loss.backward()
        run_acc += (torch.bernoulli(data).view(-1, 10).to(device=device) == v1).sum().item()

    acc = run_acc / v3.size()[0]
    if acc > best_acc:
        best_acc = acc
        
    path = "./saveMode_GBRBM_1.pth"
    torch.save(rbm_third.state_dict(), path)

    output_from_first.append(v1.tolist())
    print("1ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}\tAcc : {3}\tBest Acc : {4}" \
        .format(epoch + 1, 
                omse_loss, 
                time.time() - start,
                acc, 
                best_acc))

1ST GBrbm_first Training loss for 1 epoch 0.7179393172264099	Estimate time : 258.9570424556732	Acc : 25529.812989423077	Best Acc : 25529.812989423077
1ST GBrbm_first Training loss for 2 epoch 0.7180287837982178	Estimate time : 279.3558464050293	Acc : 25779.8305625	Best Acc : 25779.8305625
1ST GBrbm_first Training loss for 3 epoch 0.7188979387283325	Estimate time : 299.07131457328796	Acc : 26029.842927884616	Best Acc : 26029.842927884616
1ST GBrbm_first Training loss for 4 epoch 0.7179027795791626	Estimate time : 318.40238642692566	Acc : 26279.841371153845	Best Acc : 26279.841371153845
1ST GBrbm_first Training loss for 5 epoch 0.7188663482666016	Estimate time : 338.6078269481659	Acc : 26529.85330673077	Best Acc : 26529.85330673077
1ST GBrbm_first Training loss for 6 epoch 0.7187844514846802	Estimate time : 363.9108793735504	Acc : 26779.866825961537	Best Acc : 26779.866825961537
1ST GBrbm_first Training loss for 7 epoch 0.718056857585907	Estimate time : 388.6749041080475	Acc : 27029.8905

In [16]:
output_from_first = torch.tensor(output_from_first)
output_from_third = list()
print(output_from_first.size())


for epoch in range(EPOCH):
    '''Secnd gbrbm'''
    run_acc = float()
    start = time.time()
    for _, (data) in enumerate(output_from_first): 
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.normal(mean=data, std=gaussian_std).view(-1, 10).to(device=device)

        # tensor binary
        vog_second, v2 = rbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        second_train_op.zero_grad()
        omse_loss.backward()
        second_train_op.step()
        run_acc += (torch.bernoulli(data).view(-1, 10).to(device=device) == v2).sum().item()

    acc = run_acc / v2.size()[0]
    if acc > best_acc:
        best_acc = acc
    
    path = "./saveMode_GBRBM_2.pth"
    torch.save(rbm_third.state_dict(), path)

    output_from_second.append(v2.tolist())
    print("2ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}\tAcc : {3}\tBest Acc : {4}" \
        .format(epoch + 1, 
                omse_loss, 
                time.time() - start,
                acc, 
                best_acc))

C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2432/2962619170.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_from_first = torch.tensor(output_from_first)


torch.Size([50, 1040000, 10])


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_2432/2962619170.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data, dtype=torch.float32)


2ST GBrbm_first Training loss for 1 epoch 0.7181394696235657	Estimate time : 12.195174217224121	Acc : 249.97925	Best Acc : 37779.783324038464
2ST GBrbm_first Training loss for 2 epoch 0.7183735966682434	Estimate time : 12.360289096832275	Acc : 249.99392788461537	Best Acc : 37779.783324038464
2ST GBrbm_first Training loss for 3 epoch 0.7180896401405334	Estimate time : 12.222640037536621	Acc : 250.0063028846154	Best Acc : 37779.783324038464
2ST GBrbm_first Training loss for 4 epoch 0.7184255719184875	Estimate time : 12.536365985870361	Acc : 250.00674903846155	Best Acc : 37779.783324038464
2ST GBrbm_first Training loss for 5 epoch 0.7188282012939453	Estimate time : 12.344616889953613	Acc : 250.00479903846153	Best Acc : 37779.783324038464
2ST GBrbm_first Training loss for 6 epoch 0.7182480692863464	Estimate time : 12.382088899612427	Acc : 249.99823173076922	Best Acc : 37779.783324038464
2ST GBrbm_first Training loss for 7 epoch 0.7188515663146973	Estimate time : 12.319207191467285	Acc : 24

In [ ]:
output_from_second = torch.tensor(output_from_second)
run_acc = float()

for epoch in range(EPOCH):
    '''Third gbrbm'''
    for _, (data) in enumerate(output_from_second):
        start = time.time()
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, 10).to(device=device)

        vog_third, v3 = rbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3)
        
        third_train_op.zero_grad()
        omse_loss.backward()
        third_train_op.step()
        run_acc += (torch.bernoulli(data).view(-1, 10).to(device=device) == v3).sum().item()

    acc = run_acc / v3.size()[0]
    if acc > best_acc:
        best_acc = acc
        
    path = "./saveMode_GBRBM_3.pth"
    torch.save(rbm_third.state_dict(), path)

    output_from_third.append(v3.tolist())
    print("3ST GBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}\tAcc : {3}\tBest Acc : {4}" \
        .format(epoch + 1, 
                omse_loss, 
                time.time() - start,
                acc, 
                best_acc))
